In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/Othercomputers/My laptop/ML/YAGPT')

In [16]:
from config import params
from pprint import pprint
onServer = True

if not onServer:
    params['num_proc'] = 1
pprint(params)

{'additional_special_tokens': ['speaker1: ',
                               'speaker2: ',
                               'speaker3: ',
                               'speaker4: ',
                               'speaker5: ',
                               'speaker6: ',
                               'speaker7: ',
                               'speaker8: ',
                               'speaker9: ',
                               'history: '],
 'checkpoint': 'AlexWortega/instruct_rugptMedium',
 'data_path': 'encoded_texts.pkl',
 'num_proc': 5}


In [2]:
!pip install transformers datasets huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM


checkpoint = params['checkpoint']
tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token='<|pad|>')
n = 10
special_tokens = {'additional_special_tokens': params['additional_special_tokens']}
tokenizer.add_special_tokens(special_tokens)

model = AutoModelForCausalLM.from_pretrained(checkpoint)
model.resize_token_embeddings(len(tokenizer))

2023-04-02 17:22:53.857844: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-02 17:22:53.857885: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-02 17:22:55.885707: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-02 17:22:55.936176: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-02 17:22:55.938465: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic l

Embedding(50274, 1024)

In [13]:
params

{'data_path': 'encoded_texts.pkl',
 'checkpoint': 'AlexWortega/instruct_rugptMedium',
 'additional_special_tokens': ['speaker1: ',
  'speaker2: ',
  'speaker3: ',
  'speaker4: ',
  'speaker5: ',
  'speaker6: ',
  'speaker7: ',
  'speaker8: ',
  'speaker9: ',
  'history: '],
 'num_proc': 5}

In [11]:
import pickle

data_path = params['data_path']
with open(data_path, 'rb') as f:
    data = pickle.load(f)

FileNotFoundError: [Errno 2] Failed to open local file 'C:\Users\rodio\.cache\huggingface\datasets\bragovo___json\bragovo--diasum-0f75c6ea734d87f9\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-ab3c52939d35c2de.arrow'. Detail: [errno 2] No such file or directory

In [17]:
from pprint import pprint
pprint(data[0].keys())
print(len(data))

NameError: name 'data' is not defined

In [ ]:
type(data)

In [ ]:
# We need to add labels to our data
def add_labels(samples):
    samples['labels'] = samples['input_ids'].copy()
    return samples

data.map(add_labels, batched=True, num_proc=params['num_proc'])

In [ ]:
from datasets import train_test_split

data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
data_dict = data.train_test_split(test_size=0.1, shuffle=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch' # evaluation strategy to adopt during training
)

trainer = Trainer(
    model=model,                         
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['test']
)

In [ ]:
trainer.train()

In [ ]:
import math

eval_result = trainer.evaluate() # returns a dict with the evaluation metrics, cross-entropy loss and perplexity
pprint(eval_result)